### <span style="color:#FF5757">📘 Guía de Uso del módulo physiosignal</span>

Esta jupyter notebook presenta una guía paso a paso para el uso del módulo physiosignal, incluyendo explicaciones de clases, funciones clave y ejemplos de uso práctico.

✅ Recomendación previa:

🛠️ Se recomienda instalar el módulo physiosignal, incluso en modo editable, para evitar errores en las importaciones y asegurar una correcta ejecución

📦 Clases principales del módulo

* <font color="#F4B39F">Info</font> — Información general del objeto fisiológico.
* <font color="#F4B39F">Annotations</font> — Gestión de anotaciones temporales.
* <font color="#F4B39F">RawSignal</font> — Contenedor principal de señales, metadatos y anotaciones.	

🧪 Requerimientos de librerías

Para el correcto funcionamiento del módulo, se requiere tener instaladas las siguientes librerías:
* numpy>=1.21
* pandas>=1.3
* matplotlib>=3.5
* scipy>=1.9
* pyqtgraph>=0.12
* PyQt5>=5.15

#### <span style="color:#70F46C">*Importaciones requeridas para pruebas*</span>		

In [2]:
# Importaciones
from physiosignal.info import Info, Annotations
from physiosignal.signals import RawSignal
import numpy as np

# pip install -e .

### <span style="color:#FF5757">🧠 Clase Info</span>
Comencemos utilizando la clase <span style="color:#75D3FF"><strong>Info</strong></span>, la cual tiene como propósito almacenar y gestionar metadatos de registros de señales fisiológicas.

🧩 Esta clase fue diseñada para comportarse de forma similar a un diccionario (dict) de Python, permitiendo acceder y manipular metadatos de manera intuitiva.

🔑 Atributos clave
* <span style='color:#3CE28C'><strong>_ch_names:</strong></span> Lista con los nombres de los canales.
* <span style='color:#3CE28C'><strong>_ch_types:</strong></span> Tipo de cada canal (ej: 'eeg', 'ecg', etc.) o un único tipo común para todos.
* <span style='color:#3CE28C'><strong>_sfreq:</strong></span> Frecuencia de muestreo en Hz (valor por defecto: 512).

🛠️ Métodos útiles
* <span style='color:#3CE28C'><strong>visualizeInfo()</strong></span> — Visualiza la información de la instancia en forma de tabla.
* <span style='color:#3CE28C'><strong>items()</strong></span> — Devuelve todos los atributos de la instancia en formato clave-valor.
* <span style='color:#3CE28C'><strong>get()</strong></span> — Permite acceder a cualquier atributo por su nombre.

💡 Tip: Esta clase puede ser útil para inspeccionar rápidamente los metadatos de tus registros fisiológicos, validarlos y adaptarlos según el análisis que planees realizar.


In [2]:
# Hagamos pruebas
canales=[i+1 for i in range(5)]

info = Info(
 ch_names=canales,
 ch_types=["eeg"],
 bad_channels=[4],
 sfreq=512,
 register_type="Registro EEG para análisis de patrones ERDS",
 experimenter="MSc. PEREYRA Magalí",
 subject_info={"edad": 22, "sexo": "F"}
 )

#### <span style="color:#70F46C">*Testeemos los métodos*</span>		

In [3]:
info.get("ch_types")

['eeg', 'eeg', 'eeg', 'eeg', 'eeg']

In [4]:
info.items()

{'ch_names': [1, 2, 3, 4, 5],
 'ch_types': ['eeg', 'eeg', 'eeg', 'eeg', 'eeg'],
 'sfreq': 512,
 'bad_channels': [4],
 'experimenter': 'MSc. PEREYRA Magalí',
 'subject_info': {'edad': 22, 'sexo': 'F'},
 'register_type': 'Registro EEG para análisis de patrones ERDS'}

In [5]:
info.visualizeInfo()

Atributo,Datos
ch_names,"[1, 2, 3, 4, 5]"
ch_types,"['eeg', 'eeg', 'eeg', 'eeg', 'eeg']"
sfreq,512
bad_channels,[4]
experimenter,MSc. PEREYRA Magalí
subject_info,"{'edad': 22, 'sexo': 'F'}"
register_type,Registro EEG para análisis de patrones ERDS


### <span style="color:#FF5757">🧠 Clase Annotations</span>
La clase <span style="color:#75D3FF"><strong>Annotations</strong></span> tiene como propósito gestionar conjuntos de anotaciones temporales asociadas a señales fisiológicas.

🧬 Esta clase fue inspirada en la clase Annotations del paquete <a href="https://mne.tools/stable/generated/mne.Annotations.html">MNE</a>, por lo que presenta una estructura y comportamiento similares, adaptados a contextos más generales.

🔑 Atributos principales
* <span style='color:#3CE28C'><strong>onset:</strong></span> Tiempos de inicio de cada anotación.
* <span style='color:#3CE28C'><strong>duration:</strong></span> Duraciones correspondientes a cada evento.
* <span style='color:#3CE28C'><strong>description:</strong></span> Descripciones asociadas a cada anotación.
* <span style='color:#3CE28C'><strong>ch_names:</strong></span> Canales a los que se asocia cada anotación.

⚙️ Comportamiento interno

Internamente, la clase:

* Utiliza un método auxiliar del módulo <font color="#75D3FF">**_utils_**</font> para verificar y normalizar los tipos de datos recibidos (por ejemplo, convertir listas o escalares a np.ndarray).
* Ordena automáticamente todas las anotaciones por su tiempo de inicio (onset), garantizando coherencia temporal.

---

### 🧩 <span style="color:#FF5757">Explorando el submódulo utils.a_checkers</span>
Antes de avanzar con los métodos de la clase <font color="#75D3FF"><strong>Annotations</strong></font>, es útil entender qué ocurre en el proceso interno de validación y normalización de datos.
Esto se realiza mediante la función <font color="#F4B39F"><strong>_checking()</strong></font>, ubicada en el archivo a_checkers.py dentro del paquete utils/.

Esta función es invocada por el constructor de Annotations para garantizar que todos los datos estén en el formato correcto.

#### 🔄 Flujo de datos de a_checkers

La función <font color="#F4B39F">**_checking**</font> posee el siguiente flujo de datos:

✅ Conversión inicial de onset, duration y description:
* Convierte todos los parámetros a arrays unidimensionales (np.array + np.atleast_1d si aplica).
* Si algún valor es escalar o tiene longitud 1, se repite para igualar la longitud de onset.

❌ Validaciones de forma:
* Lanza ValueError si alguno de los arrays no es unidimensional.
* Lanza ValueError si las longitudes entre onset, duration, description o ch_names no coinciden.

🧠 Manejo especial de ch_names:
* Si ch_names es None: se reemplaza por un array del mismo largo que onset, con todos sus valores en None.
* Si es una lista de listas (por ejemplo, múltiples canales por anotación):
→ cada sublista se convierte en una cadena unificada por comas (ej: ['Fp1', 'Fp2'] → 'Fp1,Fp2').
* Si ch_names tiene longitud 1 y onset es más largo: se repite el valor para cada anotación.
* Si la longitud final no coincide con onset, se lanza ValueError.

📤 Salida final:
* Devuelve una tupla con onset, duration, description, ch_names como arrays uniformes, de 1D y consistentes.

In [3]:
# Importemos el módulo utils
from physiosignal.utils import a_checkers

In [4]:
# Generemos algunas variables y testeemos a_checkers
onset = [5.0, 9.0, 7.6]
description = "Evento_1"
duration = [2.0, 1.8, 2.4]
ch_names = ["Fp1", "Oz", None]
# ch_names = None

arr = a_checkers._checking(onset, duration, description, ch_names)

arr

(array([5. , 9. , 7.6]),
 array([2. , 1.8, 2.4]),
 array(['Evento_1', 'Evento_1', 'Evento_1'], dtype='<U8'),
 array(['Fp1', 'Oz', None], dtype=object))

🧠 Como vemos, la función <font color="#F4B39F"><strong>_checking()</strong></font> (desde utils/a_checkers.py) normaliza adecuadamente los parámetros de entrada convirtiéndolos en arrays unidimensionales compatibles.

⚠️ Internamente, esta función realiza varias validaciones estrictas para asegurar que los datos sean consistentes.
Una de las reglas más importantes es:

📏 La cantidad de canales (ch_names) debe coincidir con la cantidad de onsets.

Esto se debe a que cada evento en el tiempo (onset) debe estar asociado a uno o más canales, o explícitamente a None si no aplica.

💥 Veamos qué sucede si no cumplimos esta condición:

In [ ]:
# Error por longitudes diferentes
onset_1 = [3.0, 5.0, 7.6]
description_1 = "Evento_1"
duration_1 = [2.0, 1.8, 2.4]
ch_names_1 = ["Fp1", "Oz"]

# a_checkers._checking(onset_1, duration_1, description_1, ch_names_1)

🧠 Como se puede ver, el error se produce cuando hay una discrepancia entre la longitud de onset y ch_names.

✅ Sin embargo, en el caso de que se especifique solo un canal, la función <font color="#F4B39F"><strong>_checking()</strong></font> se encarga internamente de repetir ese canal tantas veces como sea necesario para que coincida con la cantidad de eventos (onsets). De esta forma, se evita que el usuario tenga que ingresar manualmente un canal por cada anotación si son todos iguales.

In [ ]:
onset_2 = [3.0, 5.0, 7.6]
description_2 = "Evento_1"
duration_2 = [2.0, 1.8, 2.4]
ch_names_2 = ["Oz"]

a_checkers._checking(onset_2, duration_2, description_2, ch_names_2)

#### 🧩 Métodos clave de la clase <font color="#75D3FF"><strong>Annotations</strong></font>
Una vez creada la instancia de esta clase, disponemos de varios métodos que nos permiten interactuar fácilmente con nuestras anotaciones:

* ✅ <span style='color:#3CE28C'><strong>add():</strong></span> Añade una nueva anotación a la instancia. Acepta los mismos parámetros que el constructor: onset, duration, description y ch_names.
* 📋 <span style='color:#3CE28C'><strong>get_annotations():</strong></span> Devuelve todas las anotaciones en formato de tabla (usualmente un DataFrame) para facilitar su visualización.
* 🔎 <span style='color:#3CE28C'><strong>find():</strong></span> Permite buscar una anotación específica según su onset, description, duration o incluso el canal asociado.

In [5]:
anotaciones = Annotations(
 onset=[5.0, 12.5, 20.0],
 duration=[2.0, 3.0, 3.5],
 description=['Inicio_Experimento', 'Evento_1', 'Evento_2'],
 ch_names=["C1", "C2", "C3"])

In [11]:
df = anotaciones.get_annotations()

for i in df.columns:
    print(type(i))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [11]:
# Método add()
anotaciones.add(onset=[30.0, 15], duration=[2.85, 1.3], description=['Evento_3', 'Evento_4'], ch_names=[["C1", "C2"], ["C1"]])

In [12]:
# Visualización de anotaciones
anotaciones.get_annotations()

,onset,duration,description,ch_names
0,5.0,2.00,Inicio_Experimento,C1
1,12.5,3.00,Evento_1,C2
2,15.0,1.30,Evento_4,C1
3,20.0,3.50,Evento_2,C3
4,30.0,2.85,Evento_3,"C1,C2"


In [16]:
# Buscando una anotación por onset
anotaciones.find((12.5))

,onset,duration,description,ch_names
1,12.5,3.0,Evento_1,C2


### 🚀<font style="color:#FF5757">Clase RawSignal</font>

Introducidas las clases <font color="#75D3FF"><strong>Annotations</strong></font> e <font color="#75D3FF"><strong>Info</strong></font>, es momento de conocer la clase principal del módulo physiosignal: <font color="#75D3FF"><strong>RawSignal</strong></font>. Esta clase representa una señal fisiológica cruda (como EEG, ECG, EMG, etc.) combinando datos de señal, metadatos de canales y anotaciones de eventos.

🔑 Atributos principales

* 📈 <span style='color:#3CE28C'><strong>data:</strong></span> Matriz de forma (n_canales, n_muestras) que contiene los datos crudos de la señal.
* ⏱️ <span style='color:#3CE28C'><strong>sfreq:</strong></span> Frecuencia de muestreo en Hz (heredada desde Info si se proporciona).
* 🧠 <span style='color:#3CE28C'><strong>info:</strong></span> Objeto de la clase Info que gestiona los metadatos del registro (nombres, tipos de canales, frecuencia, etc.).
* 📝 <span style='color:#3CE28C'><strong>anotaciones:</strong></span> Objeto de la clase Annotations, que almacena eventos y marcas temporales asociadas a la señal.
* 🔢 <span style='color:#3CE28C'><strong>first_samp:</strong></span> Índice correspondiente a la primera muestra del array data, útil en señales segmentadas.

🛠️ Métodos útiles

* 📤 <span style='color:#3CE28C'><strong>get_data():</strong></span> Permite extraer un subconjunto de datos según:
    * Intervalos de tiempo (tmin, tmax)
    * Selección de canales específicos
    * Filtro de amplitud pico-a-pico (reject_by_ptp)
* ✂️ <span style='color:#3CE28C'><strong>crop():</strong></span> Recorta la señal en un intervalo temporal determinado. No modifica el objeto original, sino que devuelve una nueva instancia de RawSignal.

* 🎯 <span style='color:#3CE28C'><strong>pick():</strong></span> Permite seleccionar uno o más canales específicos, generando una nueva instancia del objeto con solo esos canales.

* 🎚️ <span style='color:#3CE28C'><strong>filter():</strong></span> Aplica filtrado pasa-banda (definiendo l_freq, h_freq) y notch (antirruido de red) mediante SciPy.

* 📊 <span style='color:#3CE28C'><strong>plot():</strong></span> Visualización interactiva de la señal cruda con PyQtGraph, compatible con anotaciones y selección de canales.

#### 🧠<font style="color:#F4B39F">**_Señal de EEG_**</font>

In [12]:
# Testeemos RawSignal con una señal de EEG
eeg_data = np.load(r"..\tests\datos\eeg\eeg_signal.npy")
eeg_annotation = Annotations().load(path=r"..\tests\datos\eeg\eventos_ejemplo.csv")

canales=['FP1', 'FPz', 'FP2', 'AF7', 'AF3', 'AF4', 'AF8', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 
         'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 
         'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 
         'O1', 'Oz', 'O2', 'F9', 'F10']
eeg_info=Info(ch_names=canales, sfreq=512, ch_types="eeg") ##información a usar para la señal de eeg
eeg_rawsignal = RawSignal(eeg_data, info=eeg_info, anotaciones=eeg_annotation, see_log=True) #creo un objeto RawSignal

In [18]:
# Shape de los datos crudos
eeg_data.shape

(62, 388047)

#### 💪🏻<font style="color:#F4B39F">**_Señal de EMG_**</font>

In [ ]:
emg_data = np.load(r"..\tests\datos\emg\emg.npy") 
emg_annotation = Annotations().load(path=r"..\tests\datos\emg\eventos_emg.csv")
emg_info=Info(ch_names="1", sfreq=512, ch_types="ecg")

emg_rawsignal = RawSignal(emg_data.reshape(1,388971), info=emg_info, anotaciones=emg_annotation, see_log=True)

#### 🫀<font style="color:#F4B39F">**_Señal de ECG_**</font>

In [ ]:
ecg_data = np.load(r"..\tests\datos\ecg\ecg.npy") *-1
ecg_annotation = Annotations().load(path=r"..\tests\datos\ecg\eventos_ecg.csv")
ecg_info=Info(ch_names="1", sfreq=512, ch_types="ecg")

ecg_rawsignal = RawSignal(ecg_data.reshape(1,388971), info=ecg_info, anotaciones=ecg_annotation, see_log=True)


In [19]:
# Cant. de canales seleccionados
len(['FP1', 'FPz', 'FP2', 'AF7', 'AF3', 'AF4', 'AF8', 
     'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 
     'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2'])

22

#### *🟢 Uso del método <span style="color:#70F46C"><strong>get_data()</strong></span>*

Primero, echemos un vistazo a los datos que contiene nuestro objeto eeg_rawsignal, utilizando el método <span style='color:#3CE28C'><strong>get_data()</strong></span> para extraer canales específicos de la señal.

In [20]:
data, time = eeg_rawsignal.get_data(picks=['FP1', 'FPz', 'FP2', 'AF7', 'AF3', 'AF4', 'AF8', 
                                     'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 
                                     'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2'], times=True)

data

array([[   3.0584633,   69.37815  ,  512.2875   , ..., -141.60938  ,
        -154.1466   , -171.8418   ],
       [   2.483788 ,   56.613033 ,  418.65048  , ...,  -10.432716 ,
         -14.530706 ,  -20.37846  ],
       [   1.7202253,   39.700825 ,  294.78345  , ..., -111.978516 ,
        -124.08169  , -142.57555  ],
       ...,
       [   2.1442268,   52.093414 ,  391.6376   , ...,  -10.203599 ,
         -17.106434 ,  -24.769121 ],
       [   2.949064 ,   75.30474  ,  572.29645  , ...,  -12.102482 ,
         -19.469486 ,  -27.892912 ],
       [   2.442397 ,   69.35698  ,  538.45917  , ...,  -16.300808 ,
         -22.437935 ,  -29.591784 ]], shape=(22, 388047), dtype=float32)

#### *✂️ Uso del método <span style="color:#70F46C"><strong>pick()</strong></span>*

Visualizados rápidamente parte de nuestros datos, recordemos que tienen la forma (62 canales, 388047 muestras). Ahora, vamos a quedarnos únicamente con los 22 canales de interés.

Para ello, utilizaremos el método <span style="color:#3CE28C"><strong>pick()</strong></span>, el cual permite generar una nueva instancia de RawSignal que contiene solo los canales seleccionados. Esto resulta útil para reducir el tamaño de los datos o enfocar el análisis en regiones específicas.

In [21]:
pick_data = eeg_rawsignal.pick(picks=['FP1', 'FPz', 'FP2', 'AF7', 'AF3', 'AF4', 'AF8', 
                                      'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 
                                      'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2'])

#### *✂️ Recorte de señal con <span style='color:#3CE28C'><strong>crop()</strong></span>*

Una vez seleccionados los canales deseados, podemos recortar la señal a un segmento temporal específico utilizando el método <span style='color:#3CE28C'><strong>crop()</strong></span>.

In [22]:
# Quedemonos con los primeros 200 seg de muestreo (101888 muestras)

crop_data = pick_data.crop(tmin=1, tmax=200)

crop_data.data.shape

04-07-2025 17:43:16 INFO [root.crop]: Señal recortada correctamente


(22, 101888)

#### *📊 Visualización con <span style="color:#70F46C"><strong>plot()</strong></span>*

Usando el método <span style="color:#3CE28C"><strong>plot()</strong></span>, visualizaremos los primeros 10 segundos de la señal cruda, una vez realizada la selección de canales y el recorte temporal.

⚙️ El método <span style="color:#3CE28C"><strong>plot()</strong></span> está basado en PyQtGraph, lo que permite una experiencia fluida y adaptable a distintos volúmenes de datos.

In [23]:
crop_data.plot(show_anotaciones=True, duration=10)

SystemExit: 0

C:\Users\agust\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### *🔧 Filtrado de la señal con <span style="color:#70F46C"><strong>filter()</strong></span>*

Como se aprecia en la gráfica, la señal se ve bastante ruidosa, dificultando un análisis correcto.

Por ello, aplicaremos el método <span style="color:#3CE28C"><strong>filter()</strong></span>, que permite aplicar:
* Un filtro pasabanda para conservar las frecuencias de interés.
* Un filtro notch para eliminar el ruido de la red eléctrica (50/60 Hz).

⚠️ Nota: El filtro implementado no es el más óptimo, por lo que puede generar distorsiones en algunos segmentos de la señal.

In [24]:
filtered_signal = crop_data.filter(low_freq=4.0, high_freq=40.0, notch_freq=50)

#### *🔄 Visualización post-filtrado*

Filtrada la señal, veamos ahora la morfología que presenta para comprobar la reducción del ruido y la mejora en la calidad de la señal.

Usando nuevamente el método <span style="color:#3CE28C"><strong>plot()</strong></span>, podemos comparar cómo la señal cruda ruidosa ha cambiado tras aplicar el filtrado.

In [26]:
filtered_signal.plot(duration=50)

SystemExit: 0

C:\Users\agust\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### <span style="color:#FF5757">🎯 Conclusión</span>

¡Felicidades! 🎉 Has recorrido los pasos esenciales para manejar y analizar señales fisiológicas usando el módulo <font color="#F4B39F"><strong>physiosignal</strong></font>. Desde la gestión de metadatos con <span style="color:#75D3FF"><strong>Info</strong></span>, pasando por la organización de eventos con <span style="color:#75D3FF"><strong>Annotations</strong></span>, hasta el manejo y procesamiento de señales crudas con <span style="color:#75D3FF"><strong>RawSignal</strong></span>.

FIN

---